## Inverse Cooking: Recipe Generation from Food Images

In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from args import get_parser
import pickle
from model import get_model
from torchvision import transforms
from utils.output_utils import prepare_output
from PIL import Image
import time

Set ```data_dir``` to the path including vocabularies and model checkpoint

In [2]:
data_dir = ''

In [3]:
# code will run in gpu if available and if the flag is set to True, else it will run on cpu
use_gpu = True
device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
map_loc = None if torch.cuda.is_available() and use_gpu else 'cpu'


# device = torch.device('cpu')
# map_loc = None


In [4]:
print(device)

cuda


In [5]:
# code below was used to save vocab files so that they can be loaded without Vocabulary class
#ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_ingrs.pkl'), 'rb'))
#ingrs_vocab = [min(w, key=len) if not isinstance(w, str) else w for w in ingrs_vocab.idx2word.values()]
#vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_toks.pkl'), 'rb')).idx2word
#pickle.dump(ingrs_vocab, open('../demo/ingr_vocab.pkl', 'wb'))
#pickle.dump(vocab, open('../demo/instr_vocab.pkl', 'wb'))

ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'ingr_vocab.pkl'), 'rb'))
vocab = pickle.load(open(os.path.join(data_dir, 'instr_vocab.pkl'), 'rb'))

ingr_vocab_size = len(ingrs_vocab)
instrs_vocab_size = len(vocab)
output_dim = instrs_vocab_size

In [116]:
def get_ingrs(ids):
    gen_ingrs = []
    for ingr_idx in ids:
        ingr_name = chinese_vocab[ingr_idx]
        if ingr_name == '<pad>':
            break
        if ingr_name == '<end>':
            break
        gen_ingrs.append(ingr_name)
    return gen_ingrs


def mask_from_eos(ids, eos_value, mult_before=True):
    mask = torch.ones(ids.size()).to(device).byte()
    mask_aux = torch.ones(ids.size(0)).to(device).byte()

    # find eos in ingredient prediction
    for idx in range(ids.size(1)):
        # force mask to have 1s in the first position to avoid division by 0 when predictions start with eos
        if idx == 0:
            continue
        if mult_before:
            mask[:, idx] = mask[:, idx] * mask_aux
            mask_aux = mask_aux * (ids[:, idx] != eos_value)
        else:
            mask_aux = mask_aux * (ids[:, idx] != eos_value)
            mask[:, idx] = mask[:, idx] * mask_aux
    return mask
print (instrs_vocab_size, ingr_vocab_size)

23231 1488


# Load Model

In [7]:
t = time.time()
import sys; sys.argv=['']; del sys
args = get_parser()
args.maxseqlen = 15
args.ingrs_only=False
model = get_model(args, ingr_vocab_size, instrs_vocab_size)
# Load the trained model parameters
model_path = os.path.join('/home/ct2020dl5787/inversecooking/model/checkpoints', 'modelbest.ckpt')
model.load_state_dict(torch.load(model_path, map_location=map_loc))
model.to(device)
model.eval()
model.ingrs_only = False
model.recipe_only = False
model.reduction = 'none'
print ('loaded model')
print ("Elapsed time:", time.time() -t)


/home/ct2020dl5787/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


loaded model
Elapsed time: 5.803161144256592


In [8]:
for param in model.parameters():
    param.requires_grad = False

In [9]:
import copy
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [90]:
model_ft = copy.deepcopy(model.ingredient_decoder)

In [91]:
# change the last classifier
chinese_ingr_size = 353
model_ft.linear = Linear(512,chinese_ingr_size)
for param in model_ft.linear.parameters():
    param.requires_grad = True

# Load Training Data

In [115]:
import requests
from io import BytesIO
import random
from collections import Counter
import csv
vocab_file = '/home/ct2020dl5787/VireoFood172/vocab172_mapping.csv'
chinese_vocab = []
mapped_vocab = []
with open(vocab_file) as file:
    csv_reader = csv.reader(file, delimiter=',')
    for row in csv_reader:
        chinese_vocab.append(row[0])
        mapped_vocab.append(row[1])

In [13]:
true_labels = dict()
ing_labels_file = open('/home/ct2020dl5787/VireoFood172/SplitAndIngreLabel/IngreLabel.txt', 'r')
contents = ing_labels_file.readlines()
for line in contents:
    filename = line.split()[0]
    labels = line.split()[1:]
    one_hot_labels = [1 if i == '1' else 0 for i in labels]
    true_labels[filename] = one_hot_labels
ing_labels_file.close()

In [14]:
use_urls = False # set to true to load images from demo_urls instead of those in test_imgs folder
show_anyways = False #if True, it will show the recipe even if it's not valid

In [15]:
def one_hot_to_top20(one_hot_label):
    indices = [i for i, x in enumerate(one_hot_label) if x == 1]
    random.shuffle(indices)
    res = [0]*20
    res_range = min(len(indices),20)
    for i in range(res_range):
        res[i]=indices[i]
    return res

In [16]:
image_folder = os.path.join('/home/ct2020dl5787/VireoFood172/sample_pictures1720')
img_names = []
img_ingre_onehot = []
img_ingre_top20 = []
img_chinese_ingre = []
for f in os.listdir(image_folder):
    name = "/"+f
    name = name.replace("_","/",1)
    if name not in true_labels:
        print(name)
    else:
        img_names.append(f.replace("/", "_"))
        one_hot_label = true_labels[name]
        img_ingre_onehot.append(one_hot_label)
        top20_label = one_hot_to_top20(one_hot_label)
        img_ingre_top20.append(top20_label)
        chinese_ingre_name = []
        for index, label in enumerate(one_hot_label):
            if(label == 1):
                chinese_ingre_name.append(chinese_vocab[index])
        img_chinese_ingre.append(chinese_ingre_name)

/layer1.json
/layer2.json


In [17]:
# img_dict = {'img_names': img_names, 'true_onehot': img_ingre_onehot, 'true_top20':img_ingre_top20, 'ingredients_Chinese' : img_chinese_ingre}  
# true_label_df = pd.DataFrame(img_dict) 
# true_label_df.to_pickle('output/finetune_1720.pkl')

In [12]:
true_label_df = pd.read_pickle('output/finetune_1720.pkl')
true_label_df

,img_names,true_onehot,true_top20,ingredients_Chinese
0,86_10102721546691.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[345, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",[Crab]
1,140_10_14.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[15, 95, 58, 174, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[Crushed pepper, Crushed hot and dry chili, Cr..."
2,100_10_2.jpg,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[99, 7, 69, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[Sliced ham, Egg cake, Brunoise diced lentinus..."
3,64_0fd9b207887e2199159d73db9cc5a803.jpg,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[125, 16, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Seared green onion, Shredded pepper, Tofu chu..."
4,160_10_13.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[13, 146, 15, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[Black sesame, Crushed pepper, Chinese Parsley..."
...,...,...,...,...
1715,109_10_24.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[323, 21, 13, 46, 58, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Black sesame, Chinese Parsleycoriander, Groun..."
1716,84_10_2.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[27, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Green vegetables, Stinky tofu]"
1717,111_10_21.jpg,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 121, 63, 12, 58, 21, 135, 0, 0, 0, 0, 0, 0...","[Minced green onion, Hob blocks of carrot, Chi..."
1718,135_10_10.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[265, 27, 266, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[Green vegetables, Rice noodle, Fried yuba skin]"


In [13]:
#demo_files = os.listdir(image_folder)
demo_files = true_label_df['img_names'].values.tolist()

In [14]:
transf_list_batch = []
transf_list_batch.append(transforms.ToTensor())
transf_list_batch.append(transforms.Normalize((0.485, 0.456, 0.406), 
                                              (0.229, 0.224, 0.225)))
to_input_transf = transforms.Compose(transf_list_batch)


In [17]:
image_folder = os.path.join('/home/ct2020dl5787/VireoFood172/sample_pictures1720')
err_pic = []
img_file_list = []
start = time.time()
for img_file in demo_files:
    
    if img_file.endswith(".jpg"):
        image_path = os.path.join(image_folder, img_file)
        image = Image.open(image_path).convert('RGB')

        transf_list = []
        transf_list.append(transforms.Resize(256))
        transf_list.append(transforms.CenterCrop(224))
        transform = transforms.Compose(transf_list)

        image_transf = transform(image)
        image_tensor = to_input_transf(image_transf).unsqueeze(0).to(device)

#         plt.imshow(image_transf)
#         plt.axis('off')
#         plt.show()
#         plt.close()

        num_valid = 1
        img_file_list.append(img_file)
        if img_file == demo_files[0]:
            print(img_file)
            image_all = image_tensor
        else:
             image_all = torch.cat((image_all,image_tensor))


86_10102721546691.jpg


# Fine Tune

In [92]:
x_train = image_all
true_one_hot = true_label_df['true_onehot'].values.tolist()
y_train = torch.from_numpy(np.array(true_one_hot)).to(device).view(1720,1,353).float()

In [94]:
model_ft.to(device)
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model_ft.linear.parameters(), lr=0.0005)
optimizer = optim.SGD(model_ft.linear.parameters(), lr=0.05, momentum=0.8)

In [95]:
n_epochs = 10
batch_size = 30
for epoch in range(1, n_epochs+1):
    # keep track of training and validation loss
    train_loss = 0.0
    training_loss = []
    for i in range(image_all.shape[0]//batch_size + 1):
        y_batch = y_train[i*batch_size : (i+1)*batch_size]
        img_features = model.image_encoder(x_train[i*batch_size : (i+1)*batch_size])
        
        fs = img_features.size(0)
        first_word = torch.ones(fs)*0
        first_word = first_word.to(device).long()
        sampled_ids = [first_word]
        caption = torch.stack(sampled_ids, 1)
        
        optimizer.zero_grad()
        output = model_ft(ingr_features=None,ingr_mask=None,captions=caption, img_features=img_features)
        outputs = output[0]

        loss = criterion(outputs, y_batch)
        
        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    
    epoch_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', epoch_loss)

epoch: 	 1 	 training loss: 	 0.29386413919514626
epoch: 	 2 	 training loss: 	 0.09312410048883536
epoch: 	 3 	 training loss: 	 0.06830154397878153
epoch: 	 4 	 training loss: 	 0.05844607852913182
epoch: 	 5 	 training loss: 	 0.05313825902753863
epoch: 	 6 	 training loss: 	 0.049808067301737854
epoch: 	 7 	 training loss: 	 0.04751174999722119
epoch: 	 8 	 training loss: 	 0.04582214040745949
epoch: 	 9 	 training loss: 	 0.0445183080075116
epoch: 	 10 	 training loss: 	 0.04347475566740694


# Now use the new model to predict

In [96]:
y_batch = y_train[1 : 2]
img_features = model.image_encoder(x_train[1 : 2])

In [97]:
y_batch

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [98]:
fs = img_features.size(0)
first_word = torch.ones(fs)*0
first_word = first_word.to(device).long()
sampled_ids = [first_word]
caption = torch.stack(sampled_ids, 1)

output = model_ft(ingr_features=None,ingr_mask=None,captions=caption, img_features=img_features)

In [134]:
output

(tensor([[[-1.7430, -4.1452, -3.4476, -4.5136, -4.1099, -4.8991, -4.6178,
           -4.8734, -3.3767, -4.4123, -4.8999, -3.8780, -4.2914, -2.7369,
           -4.3714, -1.1093, -2.8720, -2.3560, -4.6868, -4.7516, -4.4063,
           -2.4834, -5.1058, -4.7465, -3.8479, -4.9338, -4.7815, -3.0666,
           -4.8437, -3.7044, -4.8905, -3.8066, -4.1637, -4.0668, -4.4705,
           -4.2615, -4.3102, -4.9991, -4.3461, -4.5942, -4.4687, -4.6393,
           -4.6689, -4.1176, -4.7964, -4.7339, -3.5686, -4.4925, -4.4027,
           -4.1704, -4.6039, -4.2514, -4.7652, -4.7762, -4.5847, -4.2271,
           -4.2118, -3.8170, -1.8032, -3.6881, -3.9984, -4.3286, -5.3085,
           -3.6316, -4.9489, -4.5556, -4.4226, -4.6053, -3.5010, -4.3867,
           -5.2828, -4.7405, -4.6294, -4.2804, -3.5057, -4.4628, -4.2429,
           -4.7244, -4.6494, -4.5284, -4.0796, -3.7443, -3.7941, -3.3600,
           -4.1491, -4.2733, -4.2527, -4.8450, -4.5895, -4.6027, -3.9024,
           -4.8005, -3.9925, -4.7904, 

In [100]:
output_id,output_prob = model_ft.sample(None,None,greedy = True,temperature=1.0,img_features=img_features, first_token_value=0,replacement=False)  

In [101]:
output_id

tensor([[ 15,   0,  58,   0,  17,  63, 340,  21, 157,  16,  95,  59, 135, 271,
         161, 164,  83,  54,  49, 146]], device='cuda:0')

In [117]:
##step2: run ingredient_decoder  
## batch_size could be changed

output_all = dict()
recipt_all = []
batch_size = 30


for i in range(image_all.shape[0]//batch_size + 1):
    y_batch = y_train[i*batch_size : (i+1)*batch_size]
    img_features = model.image_encoder(x_train[i*batch_size : (i+1)*batch_size])
    img_features = model.image_encoder(image_all[i*batch_size : (i+1)*batch_size])
    ingr_ids,ingr_probs = model_ft.sample(None,None,greedy = True,temperature=1.0,
                                            img_features=img_features, first_token_value=0,replacement=False)  
    sample_mask = mask_from_eos(ingr_ids, eos_value=0, mult_before=False)
    ingr_ids[sample_mask == 0] = 0

    output_all['ingr_ids'] = ingr_ids
    output_all['ingr_probs'] = ingr_probs.data
    
    ingr_ids = output_all['ingr_ids'].cpu().numpy()
    recipe = list(map(lambda x : get_ingrs(x), ingr_ids))
        
    recipt_all.extend(recipe)

In [121]:
generate_data = pd.DataFrame([img_file_list,recipt_all]).T
generate_data = generate_data.rename(columns={0: "img_names", 1: 'generate_ingre'})
generate_data

,img_names,generate_ingre
0,86_10102721546691.jpg,"[Parsley, Chinese Parsleycoriander, Dumplings,..."
1,140_10_14.jpg,"[Crushed pepper, Minced green onion, Minced gr..."
2,100_10_2.jpg,"[Minced green onion, Minced green onion, Mince..."
3,64_0fd9b207887e2199159d73db9cc5a803.jpg,"[Crushed pepper, Minced green onion, Minced gr..."
4,160_10_13.jpg,"[Minced green onion, Minced green onion, Mince..."
...,...,...
1715,109_10_24.jpg,"[Crushed pepper, Crushed hot and dry chili, Mi..."
1716,84_10_2.jpg,"[Crushed pepper, Lettuce, Minced green onion, ..."
1717,111_10_21.jpg,"[Minced green onion, Minced green onion, Mince..."
1718,135_10_10.jpg,"[Minced green onion, Minced green onion, Mince..."


In [122]:
all_info = pd.merge(true_label_df,generate_data, how='inner', on=['img_names'])
all_info.to_pickle('output/finetune_1720.pkl')

In [123]:
all_info

,img_names,true_onehot,true_top20,ingredients_Chinese,generate_ingre
0,86_10102721546691.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[345, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",[Crab],"[Parsley, Chinese Parsleycoriander, Dumplings,..."
1,140_10_14.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[15, 95, 58, 174, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[Crushed pepper, Crushed hot and dry chili, Cr...","[Crushed pepper, Minced green onion, Minced gr..."
2,100_10_2.jpg,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[99, 7, 69, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[Sliced ham, Egg cake, Brunoise diced lentinus...","[Minced green onion, Minced green onion, Mince..."
3,64_0fd9b207887e2199159d73db9cc5a803.jpg,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[125, 16, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Seared green onion, Shredded pepper, Tofu chu...","[Crushed pepper, Minced green onion, Minced gr..."
4,160_10_13.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[13, 146, 15, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[Black sesame, Crushed pepper, Chinese Parsley...","[Minced green onion, Minced green onion, Mince..."
...,...,...,...,...,...
1715,109_10_24.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[323, 21, 13, 46, 58, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Black sesame, Chinese Parsleycoriander, Groun...","[Crushed pepper, Crushed hot and dry chili, Mi..."
1716,84_10_2.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[27, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Green vegetables, Stinky tofu]","[Crushed pepper, Lettuce, Minced green onion, ..."
1717,111_10_21.jpg,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 121, 63, 12, 58, 21, 135, 0, 0, 0, 0, 0, 0...","[Minced green onion, Hob blocks of carrot, Chi...","[Minced green onion, Minced green onion, Mince..."
1718,135_10_10.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[265, 27, 266, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[Green vegetables, Rice noodle, Fried yuba skin]","[Minced green onion, Minced green onion, Mince..."


In [128]:
##step4:evalutation
def get_ingred_f1(pred, label):
    '''
    input: 
        pred: a list of predicted ingredients
        label: a list of label ingredients
    output: 
        F-1 score of the prediction
    
    i.e.  
    in: 
        pred = ["tomato", "sugar", "beef"]
        label = ["potato", "tomato"]
     
    out: 
        0.4
    '''
    intersection = list(set(pred) & set(label))
    precision = len(intersection) / len(pred)
    recall = len(intersection) / len(label)
    f1 = 2 * precision * recall / (precision + recall)
    return f1


def get_ingred_IOU(pred, label):
    '''
    input: 
        pred: a list of predicted ingredients
        label: a list of label ingredients
    output: 
        IOU of the prediction
    
    i.e.  
    in: 
        pred = ["tomato", "sugar", "beef"]
        label = ["potato", "tomato"]
     
    out: 
        0.25
    '''
    intersection = len(list(set(pred) & set(label)))
    union = len(list(set(pred) | set(label)))
    iou = intersection / union
    return iou

In [133]:
f1_summary = []
IOU_summary = []
results = all_info

for i in range(0, len(results)):
    generate_ingre = results.iloc[i]["generate_ingre"]
    reference_ingre = results.iloc[i]["ingredients_Chinese"]
    try:    
        f1_score = get_ingred_f1(generate_ingre, reference_ingre)
        iou_score = get_ingred_IOU(generate_ingre, reference_ingre)
        f1_summary.append(f1_score)
        IOU_summary.append(iou_score)
    except:
         print (i)
    
print(sum(f1_summary)/len(f1_summary))
print(sum(IOU_summary)/len(IOU_summary))

0
2
3
4
5
7
9
10
14
15
16
18
19
20
22
25
27
28
30
31
32
33
34
35
36
37
39
43
45
46
47
48
49
50
51
52
53
55
56
61
63
65
66
68
69
72
74
75
77
78
79
81
83
86
87
89
90
91
92
93
94
95
97
99
100
102
104
106
107
108
109
110
111
114
115
116
118
120
125
127
128
129
130
133
134
136
137
141
142
144
148
150
151
154
155
156
159
161
162
163
164
165
166
167
170
176
178
180
181
184
185
186
189
194
195
196
197
201
202
203
205
206
209
210
214
216
219
220
223
224
225
227
228
229
230
231
232
234
236
237
238
245
247
249
251
253
255
256
257
258
259
260
262
263
264
266
268
269
270
276
277
278
281
282
284
285
286
287
293
295
297
298
299
302
303
304
306
308
309
310
313
314
316
319
320
322
328
331
332
334
337
340
341
344
346
347
349
351
352
353
356
357
358
359
360
361
363
365
367
370
372
373
374
376
378
380
381
383
385
387
388
393
394
397
398
399
401
402
403
407
408
409
410
411
412
413
414
415
417
418
423
424
425
426
427
428
430
435
438
440
441
442
443
444
445
446
447
449
450
454
456
457
458
463
466
468
469
470